# **PySpark Ingestion + Egress + Dataloading Techniques**

In [9]:
from pyspark.sql import SparkSession
spark =  SparkSession.builder.appName("Spark-Ingress-Egress-Dataloading-Practice").getOrCreate()

print(f"[INFO] SparkSession Object Memory Reference: {spark}")

[INFO] SparkSession Object Memory Reference: <pyspark.sql.session.SparkSession object at 0xffff7c29ccd0>


## **1. Reading a CSV data and write into MySql Database using JDBC Option**

In [ ]:
#Sample customer data
"""
4000001,Kristina,Chung,55,Pilot
4000002,Paige,Chen,77,Teacher
4000003,Sherri,Melton,34,Firefighter
4000004,Gretchen,Hill,66,Computer hardware engineer
4000005,Karen,Puckett,74,Lawyer
"""
